# Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#import libraries
import numpy as np
import pandas as pd
import re
import string
import os
import math
import torch
import csv
import torch
import tensorflow as tf

from torch import tensor

In [3]:
#set variables for folder names:
datain = 'here'
dataout = 'here' #make sure dir starts with /
clinical = 'here'

# Check GPU availability

In [ ]:
#%% check if GPU is active
torch.cuda.is_available() #True
print(torch.cuda.device_count()) #1
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0)) #NVIDIA GeForce RTX 2080
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

0
Using device: cpu



# Launch transformers

In [4]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 7.9 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 27.6 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 
     |████████████████████████████████| 3.3 MB 68.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 4.7 MB/s 
     |████████████████████████████████| 1.2 MB 23.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=50e37f540b0ccf30ee3f34a4cbc886a31df6333694b16d67132305ce66d45969
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [6]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-roberta-large')
type(model) 

Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

sentence_transformers.SentenceTransformer.SentenceTransformer

# Adjacent sententence similarity

In [ ]:
df_sent = pd.read_csv(datain + 'sentence_clean.csv')
df_sent.head(5)

In [ ]:
for i in df_sent.index:
  text = df_sent['content'][i]
  text = re.sub(r'\s([?.!,"](?:\s|$))', r' ', text) #replace ' .' with ' '
  df_sent['content'][i] = text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
buglst = {}
for i,r in df_sent.iterrows():
  if r['speaker'] != 'Interviewer':
    try:
      sentence1 = r['content']
      sentence2 = df_sent['content'][i+1]

      # encode sentences to get their embeddings
      embedding1 = model.encode(sentence1, convert_to_tensor=True)
      embedding2 = model.encode(sentence2, convert_to_tensor=True)
      # compute similarity scores of two embeddings
      cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
      print("Sentence 1:", sentence1)
      print("Sentence 2:", sentence2)
      print("Similarity score:", cosine_scores.item())

      df_sent['roberta_similarity'][i+1] = cosine_scores.item()

    except KeyError:
      buglst[df_sent['Unnamed: 0'][i]] = df_sent['content'][i]
      continue
# why negatives!!!

Sentence 1: Repeat the question again 
Sentence 2: I see a woman attending to cleaning a dish while unknowingly ignoring or unawarely ignoring two children attempting to grab cookies from a pantry shelf with one of the child
Similarity score: 0.0005425326526165009


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Streaming output truncated to the last 5000 lines.
Sentence 2: no she is trying to Instead of just asking somebody she's just being so indirect about it 
Similarity score: 0.10468646883964539
Sentence 1: no she is trying to Instead of just asking somebody she's just being so indirect about it 
Sentence 2: It's kind of being eh It's kind of like being like it's too bad I have to do all the chores around here with three other people it's just kind of like Aver averting what you want to say by saying it in like a backwards ass way 
Similarity score: 0.5179811716079712
Sentence 1: It's kind of being eh It's kind of like being like it's too bad I have to do all the chores around here with three other people it's just kind of like Aver averting what you want to say by saying it in like a backwards ass way 
Sentence 2: She wants him to carry the suitcases and that she doesn't feel like she should have to do it 
Similarity score: 0.2481609582901001
Sentence 1: She wants him to carry the suitca

## add min, max, std

In [ ]:
import numpy as np
import pandas as pd

roberta = pd.read_csv('sentence_clean.csv', index_col=0)
roberta.head()

In [ ]:
roberta['sentence_count'] = ''
buglst = {}
s = 0
for i in roberta.index:
  try:
    if roberta['task'][i] == roberta['task'][i+1]:
      s += 1
    else: 
      roberta['sentence_count'][i] = s + 1
      s = 0
  except:
    buglst[roberta['Unnamed: 0.1'][i]] = roberta['content'][i]

In [9]:
roberta = roberta[['uid', 'grid', 'timepoint', 'speaker', 'task', 'roberta_similarity', 'sentence_count']]

In [10]:
roberta.columns

Index(['uid', 'grid', 'timepoint', 'speaker', 'task', 'roberta_similarity',
       'sentence_count'],
      dtype='object')

In [12]:
roberta = roberta[roberta.timepoint == 'BL']
roberta = roberta[roberta.task.isin(['AboutYourself', 'HowsItGoing'])]

In [15]:
roberta = roberta.reset_index(drop=True)
len(roberta.uid.unique())

76

In [ ]:
clinical = pd.read_csv('data_analysis.csv', index_col=0)
clinical.head()

In [ ]:
roberta['sentence_count'] = pd.to_numeric(roberta['sentence_count'], errors='coerce')
similarity_sentence_clean_gt = roberta.groupby(['grid']).agg({ 
                                            "roberta_similarity": ['min', 'max', 'std']
                                            })

similarity_sentence_clean_gt.sort_values(by='grid', ascending=True, inplace=True)
similarity_sentence_clean_gt.head()

In [ ]:
final = pd.merge(similarity_sentence_clean_gt, clinical, on='grid', how='left')
final.head()

In [24]:
final = final.rename(columns = {"('roberta_similarity', 'min')": "roberat_similarity_min",
"('roberta_similarity', 'max')": "roberta_similarity_max", "('roberta_similarity', 'std')": "roberta_similarity_sd"})

## add mean

In [50]:
df_clinical = pd.read_csv(clinical + 'data_analysis.csv')
df_sent['sentence_count'] = pd.to_numeric(df_sent['sentence_count'], errors='coerce')
similarity_sentence_clean_gt = df_sent.groupby(['grid', 'task']).agg({ 
                                            "roberta_similarity_sum": lambda x: x.sum(skipna=True),
                                            'sentence_count': lambda x: x.sum(skipna=True)
                                            })

similarity_sentence_clean_gt['similarity_mean'] = similarity_sentence_clean_gt['roberta_similarity_sum'] / (similarity_sentence_clean_gt['sentence_count']/2)

similarity_sentence_clean_gt.sort_values(by='grid', ascending=True, inplace=True)
